# TODO
- ładnie opisać co się dzieje
- ładnie na funkcje podzielić i refactor ogólnie
- obsługa inputu- obsługa plakatów <3

In [177]:
import numpy as np
import pandas as pd
import requests
import itertools

## Move API

In [178]:
api_key = '63a56440'
url = f'http://www.omdbapi.com/?apikey={api_key}'
params = {"t":"", "plot": "short", "r":"json"}

In [179]:
def fetch_movies(movies):
    movies_data = {}
    for movie in movies:
        params["t"] = movie
        response = requests.get(url, params=params).json()
        movies_data[movie] = response
    return movies_data

## Get input from user

In [180]:
value = None
movies = []

print("Please enter a movie title to compare. When you finish press enter.\n")

while value != '':
    value = input()
    if value != '':
        movies.append(value)

movies_data = fetch_movies(movies)

Please enter a movie title to compare. When you finish press enter.

die hard
interstellar
love actually



In [185]:
criteria = list(movies_data[movies[0]].keys())

for index, criterium in enumerate(criteria[1:]):
    print(f'{index + 1}. {criterium}')

value = None
chosen_criteria = []

print("\nPlease enter a criteria number to compare. When you finish press enter.\n")

while value != '':
    value = input()
    if value != '':
        chosen_criteria.append(criteria[int(value)])

1. Year
2. Rated
3. Released
4. Runtime
5. Genre
6. Director
7. Writer
8. Actors
9. Plot
10. Language
11. Country
12. Awards
13. Poster
14. Ratings
15. Metascore
16. imdbRating
17. imdbVotes
18. imdbID
19. Type
20. DVD
21. BoxOffice
22. Production
23. Website
24. Response

Please enter a criteria number to compare. When you finish press enter.

8
12
17



## Create pairs to compare

In [186]:
movies_pairs = list(itertools.combinations(movies, 2))
criteria_pairs = list(itertools.combinations(chosen_criteria, 2))

print("Movies pairs: ", movies_pairs)
print("\nCriteria pairs: ", criteria_pairs)

Movies pairs:  [('die hard', 'interstellar'), ('die hard', 'love actually'), ('interstellar', 'love actually')]

Criteria pairs:  [('Actors', 'Awards'), ('Actors', 'imdbVotes'), ('Awards', 'imdbVotes')]


## User rating movies

In [187]:
movies_matrices = [pd.DataFrame(columns=movies, index=movies) for _ in range(len(chosen_criteria))]
criteria_matrices = pd.DataFrame(columns=chosen_criteria, index=chosen_criteria)


for i, criterium in enumerate(chosen_criteria):
    for pair in movies_pairs:
        matrix = movies_matrices[i]
        np.fill_diagonal(matrix.values, 1)
        
        val =  float(input(f'\nCriterium: {criterium}\n    {pair[0]}: {movies_data[pair[0]][criterium]}\n    {pair[1]}: {movies_data[pair[1]][criterium]}\n Compare first to second:\n'))
        matrix[pair[0]][pair[1]] = val
        matrix[pair[1]][pair[0]] = 1/val



Criterium: Actors
    die hard: Bruce Willis, Alan Rickman, Bonnie Bedelia
    interstellar: Matthew McConaughey, Anne Hathaway, Jessica Chastain
 Compare first to second:
2

Criterium: Actors
    die hard: Bruce Willis, Alan Rickman, Bonnie Bedelia
    love actually: Hugh Grant, Martine McCutcheon, Liam Neeson
 Compare first to second:
1

Criterium: Actors
    interstellar: Matthew McConaughey, Anne Hathaway, Jessica Chastain
    love actually: Hugh Grant, Martine McCutcheon, Liam Neeson
 Compare first to second:
0.3

Criterium: Awards
    die hard: Nominated for 4 Oscars. 8 wins & 6 nominations total
    interstellar: Won 1 Oscar. 44 wins & 148 nominations total
 Compare first to second:
0.1

Criterium: Awards
    die hard: Nominated for 4 Oscars. 8 wins & 6 nominations total
    love actually: 10 wins & 29 nominations total
 Compare first to second:
4

Criterium: Awards
    interstellar: Won 1 Oscar. 44 wins & 148 nominations total
    love actually: 10 wins & 29 nominations total


## User rating criteria priority

In [188]:
for pair in criteria_pairs:
    np.fill_diagonal(criteria_matrices.values, 1) 

    val =  float(input(f'Rate importance: {pair[0]} to {pair[1]}:\n'))
    criteria_matrices[pair[0]][pair[1]] = val
    criteria_matrices[pair[1]][pair[0]] = 1/val

Rate importance: Actors to Awards:
0.5
Rate importance: Actors to imdbVotes:
10
Rate importance: Awards to imdbVotes:
10


## EVM ranking method

In [189]:
def count_evm(comparison_matrix):
    matrix = comparison_matrix.to_numpy(dtype=float)
    values, vectors = np.linalg.eigh(matrix)
    max_index = np.argmax(values)
    v = np.abs(vectors[:, max_index])
    return v / np.sum(v)

In [190]:
priorities = pd.DataFrame(columns=movies, index=chosen_criteria)
criteria_importance = pd.DataFrame(columns=chosen_criteria)

In [191]:
# count priorities and importance
# multiply priorities * importance
# sum up
criteria_matrices

,Actors,Awards,imdbVotes
Actors,1,2.0,0.1
Awards,0.5,1,0.1
imdbVotes,10.0,10.0,1


In [192]:
criteria_priorities = pd.DataFrame(columns=chosen_criteria)
criteria_priorities.loc[0] = count_evm(criteria_matrices)
criteria_priorities = criteria_priorities.rename(index={0: "priority"})

comparison_priorities = pd.DataFrame(columns=chosen_criteria, index=movies)
for i, criterium in enumerate(chosen_criteria):
    matrix = movies_matrices[i]
    comparison_priorities[criterium] = count_evm(matrix)   

In [193]:
comparison_priorities

,Actors,Awards,imdbVotes
die hard,0.407799,0.092022,0.333333
interstellar,0.372587,0.449549,0.333333
love actually,0.219615,0.458429,0.333333


In [194]:
criteria_priorities

,Actors,Awards,imdbVotes
priority,0.295034,0.295034,0.409931


In [195]:
for criterium in chosen_criteria:  
    comparison_priorities[criterium] = comparison_priorities[criterium].values * criteria_priorities[criterium].values
comparison_priorities

,Actors,Awards,imdbVotes
die hard,0.120315,0.027150,0.136644
interstellar,0.109926,0.132633,0.136644
love actually,0.064794,0.135252,0.136644


In [196]:
comparison_priorities["Goal"] = comparison_priorities.sum(axis=1)
comparison_priorities

,Actors,Awards,imdbVotes,Goal
die hard,0.120315,0.027150,0.136644,0.284108
interstellar,0.109926,0.132633,0.136644,0.379202
love actually,0.064794,0.135252,0.136644,0.336690


# ---------------------------------------------------------------------------------------------------------------

dataframe = pd.read_csv("piwo.csv", index_col=0, sep=' ')

In [ ]:
dataframe

In [ ]:
dataframe.columns.values.tolist()

In [ ]:
dataframe[["Koszt", "Procent"]]

In [ ]:
dataframe["Koszt"]

In [ ]:
def get_columns_names(dataframe):
    return dataframe.columns.values.tolist()

In [ ]:
print(get_columns_names(dataframe))

In [ ]:
def filter_dataframe_columns(dataframe, columns_to_stay):
    dataframe_columns_names = get_columns_names(dataframe)
    columns_to_drop = set(dataframe_columns_names) - set(columns_to_stay)
    return dataframe.drop(columns=columns_to_drop)

In [ ]:
only_good_values = filter_dataframe_columns(dataframe, ["Koszt", "Procent", "Smak"])
only_good_values

In [ ]:
def count_evm(comparison_matrix):
    matrix = comparison_matrix.to_numpy(dtype=float)
    values, vectors = np.linalg.eigh(matrix)
    max_index = np.argmax(values)
    v = np.abs(vectors[:, max_index])
    return v / np.sum(v)

In [ ]:
comp_matrix = np.array([[1,2, 9],[1/2,1, 2], [1/9, 1/2, 1]])
print(comp_matrix)
w, v = count_eigenvalue(comp_matrix)
print(w)
print(v)

In [ ]:
def get_final_ranking(dataframe, weights):
    data = dataframe.to_numpy()
    return np.sum(data*weights, axis=-1)

In [ ]:
print(make_ranking(only_good_values, w))

In [ ]:
def apply_ranking_to_dataframe(dataframe, weigth):
    result_dataframe = dataframe.copy(deep=True)
    ranking = get_final_ranking(dataframe, weigth)
    result_dataframe["Wyniki"] = ranking.tolist()
    result_dataframe = result_dataframe.sort_values("Wyniki", ascending=False)
    return result_dataframe

In [ ]:
result_dataframe = apply_ranking_to_dataframe(only_good_values, w)
result_dataframe